In [78]:
import pandas as pd
import json
from scipy import stats
import numpy as np
import re
import csv

In [4]:
## keyword
read_path = '../csv/search_click_200419_200425.csv'
raw_keyword = pd.read_csv(read_path)
raw_keyword.head()

,ref_term,name,click
0,닌텐도 스위치,디지털/가전,401150
1,아이폰,디지털/가전,130835
2,노트북,디지털/가전,125017
3,에어팟,디지털/가전,115235
4,닌텐도 스위치 라이트,디지털/가전,108723


In [9]:
raw_keyword['click'].sum()

48762211

In [10]:
raw_keyword.groupby(['name'], as_index=False).agg({'click': 'sum'})

,name,click
0,(구)패션의류,3
1,구인구직,38434
2,기타,1380744
3,남성의류,6122766
4,도서/티켓/취미/애완,3223692
5,디지털/가전,10842509
6,번개나눔,84139
7,뷰티/미용,1461129
8,생활/문구/가구/식품,2188223
9,스타굿즈,2885185


In [125]:
digital = raw_keyword[raw_keyword['name'] == '디지털/가전']
digital_1000 = digital.head(1000)
# save
save_path = '../csv/result/digital_1000.csv'
digital_1000.to_csv(save_path, index=False, mode='w', header=True)

In [110]:
## keyword
read_path = '../csv/digital_1000_naver_scrap_result.csv'
digital_naver = pd.read_csv(read_path)
digital_naver = digital_naver[(digital_naver['filter_value'] != '-') & 
                              (digital_naver['filter_value'] != '정확한 검색어 인지 확인하시고 다시 검색해 주세요.')] # 결과 없는 row 제거
digital_naver = digital_naver[(digital_naver['filter_key'] != '가격') &
                              (digital_naver['filter_key'] != '배송/혜택/색상') & 
                              (digital_naver['filter_key'] != '쇼핑연관')] # 가격, 배송/혜택/색상, 쇼핑연관 key 제거
digital_naver.tail()

,original_keyword,keyword_query,filter_key,filter_value,search_count
6117,lg g6,lg+g6,내장메모리,"['128GB', '4GB', '32GB', '64GB']",1530
6121,마리오파티,마리오파티,카테고리,"['게임타이틀', '휴대용게임기']",1336
6122,마리오파티,마리오파티,브랜드,"['닌텐도', '노브랜드', '파티해', '아트박스', 'OST', '바보사랑', ...",1336
6126,gtx1070ti,gtx1070ti,카테고리,"['NVIDIA계열', 'AMD계열']",1335
6127,gtx1070ti,gtx1070ti,브랜드,"['MSI', '조텍', 'ASUS', '기가바이트']",1335


In [111]:
filter_keys =[]
def get_filter_key(row):
    key = row['filter_key']
    if key not in filter_keys:
        filter_keys.append(key)
    return None

digital_naver[['filter_key']].apply(get_filter_key, axis=1)

filters = {key: [] for key in filter_keys}

def get_filter_values(row):
    key = row['filter_key']
    values = row['filter_value'].strip("[").strip("]").replace("'","").split(", ")
    
    filters_value = filters[key]
    
    for value in values:
        if value not in filters_value:
            filters[key].append(value)
    return None

digital_naver[['filter_key', 'filter_value']].apply(get_filter_values, axis=1)

1       None
2       None
3       None
7       None
8       None
        ... 
6117    None
6121    None
6122    None
6126    None
6127    None
Length: 3219, dtype: object

In [72]:
file = open('../csv/result/digital_1000_named_entity.csv', mode="w")
writer = csv.writer(file)
writer.writerow(["named_entity", "name"])

for key in filters.keys():
    result =[]
    for column in filters[key]:
        result = [key, column]
        writer.writerow(result)

In [2]:
## import name-entity file
read_path = '../csv/name_entity_digital.csv'
name_entity_digital = pd.read_csv(read_path)
name_entity_digital = name_entity_digital[name_entity_digital['entity'] != '키워드추천']

name_entity_digital_dict = name_entity_digital.set_index('name')['entity'].to_dict()


def get_word (sentence):
    sentence_raw = sentence[0]
    sentence_no_space = sentence_raw.replace(" ", "")
    
    if sentence_raw in name_entity_digital_dict.keys():
        entity = name_entity_digital_dict[sentence_raw]
        return {"keyword": sentence_raw, "keyword_query": sentence_no_space, "entity": entity}
    elif sentence_no_space in name_entity_digital_dict.keys():
        entity = name_entity_digital_dict[sentence_no_space]
        return {"keyword": sentence_raw, "keyword_query": sentence_no_space, "entity": entity}
    else:
        words = sentence_raw.split()
        entity = []
        for word in words:
            if word in name_entity_digital_dict.keys():
                entity.append(name_entity_digital_dict[word])
            else:
                entity.append('not_matched')
        return {"keyword": sentence_raw, "keyword_query": words, "entity": entity}


keyword_entity = digital_1000[['ref_term']].apply(get_word, axis=1)


file = open('../csv/result/keyword_entity_digital.csv', mode="w")
writer = csv.writer(file)
writer.writerow(["keyword", "keyword_query", "entity"])

for keyword in keyword_entity:
    result = []
    for keys in keyword.keys():
        result.append(keyword[keys])
    writer.writerow(result)

NameError: name 'digital_1000' is not defined

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,ref_term,click
76448,닌텐도스위치,501332
76825,닌텐도스위치라이트,133832
144904,아이폰,130932
72384,노트북,125658
163857,에어팟프로,124078
...,...,...
126632,서든닉네임,1302
40775,s10+,1301
121241,사운드블라스터,1299
191811,카세트,1299


In [88]:
digital_refined = digital.groupby(['reterm'], as_index=False).agg({'click': 'sum'})
digital_refined
# digital_1000 = digital__.head(1000)
# digital_1000.head(30)

,term,click
0,,92
1,\n\n보만EK-6130\n\n,3
2,\n14t990-ga50k,2
3,\nTITAN,11
4,!.,1
...,...,...
219225,힙합플레이야,1
219226,￼나이키,1
219227,𝐟𝐥𝐚𝐱롤,7
219228,🌸,1


In [126]:
word_count = {}

def get_word_from_words(words):
    word_list =[]
    for word in words.split(' '):
        word_list.append(word)
    return word_list

def get_words(row):
    ref_terms = row[0]
    ref_term = get_word_from_words(ref_terms)
    click = row[2]
    for term in ref_term:
        if term in word_count.keys():
            word_count[term] = word_count[term] + click
        else:
            word_count[term] = click
    return word_count

# digital_word = get_words(digital_1000)
digital_1000.apply(get_words, axis=1)
word_count

{'닌텐도': 844809,
 '스위치': 774801,
 '아이폰': 303902,
 '노트북': 149879,
 '에어팟': 268427,
 '라이트': 154969,
 '동물의숲': 204062,
 '에디션': 128137,
 '닌텐도스위치': 129540,
 '아이폰8': 113182,
 '프로': 176869,
 '': 202107,
 '아이폰se': 116253,
 '아이패드': 325437,
 '컴퓨터': 91418,
 '아이폰11': 90634,
 '아이폰x': 74777,
 '에어팟2': 88854,
 '아이폰xs': 95062,
 '아이폰7': 71616,
 '에어3': 60591,
 '애플워치': 45894,
 '맥북': 49657,
 '모니터': 65668,
 '아미보': 54411,
 '에어팟프로': 44648,
 'se': 37129,
 '아이폰xr': 41774,
 '노트10': 36013,
 '갤럭시s10': 34846,
 '3세대': 33383,
 '롤': 107225,
 '계정': 62081,
 '오버워치': 81180,
 '아이폰11pro': 42797,
 '버즈': 85209,
 '노트8': 30317,
 '노트9': 30967,
 '게이밍노트북': 25473,
 '7세대': 29069,
 '갤럭시': 117164,
 '플러스': 44607,
 '러브비트': 32631,
 '스페이스그레이': 23655,
 '아이폰6': 25167,
 'ps4': 34569,
 '그래픽카드': 24285,
 '플스4': 36224,
 '애플펜슬': 43729,
 '아미보카드': 22537,
 '아이폰8플러스': 24111,
 '맥북프로': 34428,
 '갤럭시탭': 24432,
 '아이폰6s': 23773,
 '갤럭시s20': 17662,
 '냉장고': 17502,
 '피파4': 21494,
 '모동숲': 32721,
 '신형': 22291,
 '갤럭시탭s6': 17130,
 '동물의숲에디션': 30760,
 's10': 27693,
 '태

dict_keys(['노트북', '마우스', '올인원pc', '컴퓨터', '모니터', '필름카메라', '본체', '에어프라이어', '전자레인지', 'cpu', '제습기', '복합기', '테블릿', '미니냉장고', '안마의자', '밥솥', '폴더폰', '건조기', '미니빔', '냉동고', '제빙기', '중고컴퓨터', '하드디스크', '컴퓨터케이스', '캠핑', '일렉기타', '게이밍노트북', 'tv', '세탁기', '컴퓨터본체', 'ssd', '냉장고', '게이밍컴퓨터', '핸드폰', '오븐', '게이밍마우스', '게이밍', '청소기', '메인보드', '외장하드', '카메라', '게이밍모니터'])


In [147]:
## keyword
import_path = '../csv/search_click_200419_200425.csv'
raw_keyword = pd.read_csv(import_path)

digital = raw_keyword[raw_keyword['name'] == '디지털/가전']

digital['ref_term'].replace(" ", "", regex=True, inplace=True)

digital = digital.groupby(['ref_term'], as_index=False).agg({'click': 'sum'})
digital_1000_no_space = digital.sort_values('click', ascending=False).head(1000)

# save
save_path = '../csv/digital_1000_no_space.csv'
digital_1000_no_space.to_csv(save_path, index=False, mode='w', header=True)

digital_1000_no_space.head()

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,ref_term,click
76448,닌텐도스위치,501332
76825,닌텐도스위치라이트,133832
144904,아이폰,130932
72384,노트북,125658
163857,에어팟프로,124078


In [148]:
## import category
import_path = '../csv/keyword_category.csv'
keyword_category = pd.read_csv(import_path)

keyword_category_dict = keyword_category.set_index('name')['entity'].to_dict()

## import category
import_path = '../csv/keyword_etc.csv'
keyword_etc = pd.read_csv(import_path)

keyword_etc_dict = keyword_etc.set_index('name')['entity'].to_dict()

def search_category(row):
    keyword = row['ref_term']
    categories = keyword_category_dict.keys()
    
    for category in categories:
    
        if re.search(category, keyword):
            return re.search(category, keyword).group()
        
def search_etc(row):
    keyword = row['ref_term']
    etcs = keyword_etc_dict.keys()
    
    for etc in etcs:
    
        if re.search(etc, keyword):
            return re.search(etc, keyword).group()

    
digital_1000_no_space['category'] = digital_1000_no_space.apply(search_category, axis=1)
digital_1000_no_space['etc'] = digital_1000_no_space.apply(search_etc, axis=1)


digital_1000_no_space

# save
save_path = '../csv/result/digital_1000_no_space_result.csv'
digital_1000_no_space.to_csv(save_path, index=False, mode='w', header=True)

In [88]:
## keyword
import_path = '../csv/search_click_200419_200425.csv'
raw_keyword = pd.read_csv(import_path)

digital = raw_keyword[raw_keyword['name'] == '디지털/가전']

digital['ref_term'].replace(" ", "", regex=True, inplace=True)

digital = digital.groupby(['ref_term'], as_index=False).agg({'click': 'sum'})
digital_200_no_space = digital.sort_values('click', ascending=False).head(200)

# save
save_path = '../csv/digital_200_no_space.csv'
digital_200_no_space.to_csv(save_path, index=False, mode='w', header=True)

digital_200_no_space.head()

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,ref_term,click
76448,닌텐도스위치,501332
76825,닌텐도스위치라이트,133832
144904,아이폰,130932
72384,노트북,125658
163857,에어팟프로,124078


In [55]:
## import series
import_path = '../csv/digital_series.csv'
df_digital_series = pd.read_csv(import_path)

digital_series =[]

def find_series(row):
    parent = row[0]
    name = row[1]
    if re.match(parent, name):
        digital_series.append(name)
    else: 
        digital_series.append(name)
        digital_series.append(str(parent+name))
    return None

df_digital_series.apply(find_series, axis=1)

print(digital_series)

['액티브2', '갤럭시워치액티브2', '갤럭시폴드', '고프로7', '고프로8', '노트', '갤럭시노트', '노트10', '갤럭시노트10', '노트2', '갤럭시노트2', '노트3', '갤럭시노트3', '노트4', '갤럭시노트4', '노트5', '갤럭시노트5', '노트6', '갤럭시노트6', '노트7', '갤럭시노트7', '노트8', '갤럭시노트8', '노트9', '갤럭시노트9', 'a90', '갤럭시a90', '닌텐도2ds', '닌텐도동물의숲', '스위치닌텐도동물의숲', '닌텐도스위치', '스위치닌텐도스위치', '닌텐도스위치동물의숲에디션', '스위치닌텐도스위치동물의숲에디션', '닌텐도스위치라이트', '스위치닌텐도스위치라이트', '닌텐도dslite', '모여봐요동물의숲', '닌텐도모여봐요동물의숲', '아이팟클래식', '아이패드7', '아이패드7세대', '아이패드미니', '아이패드에어', '아이패드프로2', '아이패드프로3', '아이폰11프로', '아이폰11pro', '아이폰12', '아이폰6s플러스', '아이폰7플러스', '아이폰8', '아이폰8+', '아이폰se', '아이폰se2', '아이폰xr', '아이폰xs', '애플워치4', '애플워치5', '애플펜슬2세대', '에어팟프로', '폴드', '갤럭시폴드', '플레이스테이션4', '피파온라인4', 'z플립', '갤럭시z플립']


In [87]:
def find_entity(row):
    ref_term = row['ref_term']
    click = row['click']
    for series in digital_series:
        if re.search(series, ref_term):
            entity = '시리즈'
            remained_term = ref_term.replace(series, '')
            return series
    


digital_200_no_space['시리즈'] = digital_200_no_space.apply(find_entity, axis=1)

digital_200_no_space['remained'] = digital_200_no_space['ref_term'].replace(digital_200_no_space['시리즈'], '')

digital_200_no_space

,ref_term,click,시리즈,remained
76448,닌텐도스위치,501332,닌텐도스위치,
76825,닌텐도스위치라이트,133832,닌텐도스위치,닌텐도스위치라이트
144904,아이폰,130932,None,아이폰
72384,노트북,125658,노트,노트북
163857,에어팟프로,124078,에어팟프로,
...,...,...,...,...
56483,갤럭시워치액티브2,6279,액티브2,갤럭시워치액티브2
149399,아이폰8256,6257,아이폰8,아이폰8256
147660,아이폰6s플러스,6208,아이폰6s플러스,
88235,라이젠,6158,None,라이젠


76448       (닌텐도스위치, 시리즈)
76825       (닌텐도스위치, 시리즈)
144904               None
72384           (노트, 시리즈)
163857       (에어팟프로, 시리즈)
               ...       
56483         (액티브2, 시리즈)
149399        (아이폰8, 시리즈)
147660    (아이폰6s플러스, 시리즈)
88235                None
141743      (아이팟클래식, 시리즈)
Length: 200, dtype: object